In [33]:
%reset -f

In [34]:
import pandas as pd
from sqlalchemy import create_engine
import os
from time import time

In [35]:
# sql eng params
user = 'root'
password = 'root'
host = 'localhost'
port = '5432'
db = 'ny_taxi_0'
db_tablename = 'KONTOL'
url = 'https://raw.githubusercontent.com/ryandipp/nytaxidatapipeline/main/datasets/yellowtaxitrip_0121.csv'

engaddress= f"postgresql://{user}:{password}@{host}:{port}/{db}"
psqlengine= create_engine(engaddress)

In [36]:
def connect(engine):
    engine.connect()
    print("CONNECTION SUCCESSFUL")

connect(psqlengine)

CONNECTION SUCCESSFUL


In [37]:
def wget(url):
    filename = url.split('/')[-1]
    os.system(f"curl {url} -o {filename}")
    print(f"{filename} downloaded successfully!")
    return filename

filename= wget(url)

yellowtaxitrip_0121.csv downloaded successfully!


##Extracting CSV to Postgres

In [31]:
#schema= pd.read_csv('yellowtaxitrip_0121.csv', nrows=1)
#print(
#    pd.io.sql.get_schema(schema, name='yellowtaxidata', con=psqlengine))

In [38]:
filename

'yellowtaxitrip_0121.csv'

In [40]:
def mkschema(filename):
    data=pd.read_csv(filename, nrows=1)
    try:
        for c in data.columns:
            if 'date' in c:
                data[c]=pd.to_datetime(data[c])
        data.head(0).to_sql(name=db_tablename, con=psqlengine, if_exists='replace')
        print("schema created")
    except Exception as e:
        print(f"failed: {e}")

mkschema(filename)


schema created


In [41]:
chunk_size=20000
df_iter = pd.read_csv(filename, iterator=True, chunksize=chunk_size, low_memory=False, nrows=30000)
nrows = len(pd.read_csv(filename, usecols=[1], low_memory=False))

In [42]:
nchunck= nrows//chunk_size+1
for i, df in enumerate(df_iter, start=1):
    
    tstart= time()
        
    df.to_sql(name=db_tablename, con=psqlengine, if_exists='append')
    
    tend= time()
    
    print(f"Inserted chunk {i} of {nchunck}... took {tend-tstart:.3} secs")

Inserted chunk 1 of 14... took 2.33 secs
Inserted chunk 2 of 14... took 1.07 secs


In [18]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
260000,2,2021-01-08 08:48:13,2021-01-08 08:56:48,1,1.38,1,N,142,238,1,8.0,0.0,0.5,2.26,0.0,0.3,13.56,2.5
260001,1,2021-01-08 08:34:19,2021-01-08 08:48:07,1,2.60,1,N,262,163,1,11.5,2.5,0.5,1.00,0.0,0.3,15.80,2.5
260002,2,2021-01-08 08:05:27,2021-01-08 08:11:54,2,1.68,1,N,234,231,1,7.0,0.0,0.5,2.06,0.0,0.3,12.36,2.5
260003,2,2021-01-08 08:13:45,2021-01-08 08:17:41,2,1.72,1,N,231,234,1,6.5,0.0,0.5,1.96,0.0,0.3,11.76,2.5
260004,2,2021-01-08 08:30:13,2021-01-08 08:42:21,2,2.38,1,N,113,161,1,10.5,0.0,0.5,1.38,0.0,0.3,15.18,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267808,2,2021-01-08 11:34:17,2021-01-08 11:53:59,1,5.80,1,N,137,40,1,19.5,0.0,0.5,2.28,0.0,0.3,25.08,2.5
267809,1,2021-01-08 11:41:30,2021-01-08 11:48:32,1,1.70,1,N,262,74,1,7.0,2.5,0.5,2.50,0.0,0.3,12.80,2.5
267810,1,2021-01-08 11:54:56,2021-01-08 12:00:29,1,0.70,1,N,263,236,2,5.5,2.5,0.5,0.00,0.0,0.3,8.80,2.5
267811,1,2021-01-08 11:06:42,2021-01-08 11:09:31,1,0.60,1,N,43,142,1,4.5,2.5,0.5,1.00,0.0,0.3,8.80,2.5


In [17]:
df.describe().columns

Index(['VendorID', 'passenger_count', 'trip_distance', 'RatecodeID',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge'],
      dtype='object')